In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=C:/Users/cherp/AppData/Local/Google/Chrome/User Data/Default')

driver = webdriver.Chrome(options=options)

url = 'https://today.sberbankvmeste.ru/npo'
driver.get(url)

try:
    while True:
        driver.execute_script("window.scrollBy(0, 100);")
        time.sleep(2)
        # Находим кнопку "Показать ещё"
        show_more_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//button[@class='sc-DTJrX kqXKXN sc-cvbbAY iOBEtp'][@value='Показать ещё']"))
        )
        actions = ActionChains(driver)
        actions.move_to_element(show_more_button).click().perform()
        
except TimeoutException:
    # Как только кнопка "Показать ещё" исчезнет
    print("Кнопка 'Показать ещё' больше не найдена. Работа завершена.")

soup = BeautifulSoup(driver.page_source, 'html.parser')

elements = soup.find_all('a', class_='sc-cBOTKl dyhqtK')

prefix = "https://today.sberbankvmeste.ru"

# Извлечение и преобразование ссылки
links = [urljoin(prefix, element['href']) for element in elements]

with open('Ссылки_фондыСберВместе.txt', 'w') as file:
    for link in links:
        file.write(link + '\n')

driver.quit()


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=C:/Users/cherp/AppData/Local/Google/Chrome/User Data/Default')

driver = webdriver.Chrome(options=options)

texts_dict = {}

def extract_numbers(text):
    numbers = re.findall(r'\d+', text)
    if numbers:
        return int(''.join(numbers))
    return None

for link in tqdm(links):
    driver.get(link)
    time.sleep(5)

    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    main_text = soup.find('div', class_='sc-OqFzE jaZUqK').get_text()
    nested_blocks = soup.find_all('div', class_='sc-dcOKER lcypPc')
    
    main_text_dict = {}
    
    for block in nested_blocks:
        key = block.find('div', class_='sc-fAMDQA cQyKGc').get_text()
        value = block.find('div', class_='sc-ibnDSj iVMKpT').get_text()
        numeric_value = extract_numbers(value)
        if numeric_value is not None:
            main_text_dict[key] = numeric_value
        else:
            main_text_dict[key] = value
    
    texts_dict[main_text] = main_text_dict
    texts_dict[main_text]["Ссылка на страницу"] = link

driver.quit()

df = pd.DataFrame.from_dict(texts_dict, orient='index')
df = df[['Собрано фондом', 'Человек помогло фонду', 'Завершённых сборов', 'Ссылка на страницу']]
df = df.sort_values(by='Собрано фондом', ascending=False)
df.to_excel('Информация_фондыСберВместе.xlsx', index_label='Название фонда')

print(df)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=C:/Users/cherp/AppData/Local/Google/Chrome/User Data/Default')

driver = webdriver.Chrome(options=options)
driver.get("https://nuzhnapomosh.ru/funds/")

# Найдем чекбокс "Только верифицированные"
checkbox = driver.find_element(By.XPATH, '//span[@class="np-form-checkbox__title" and @checked=""]')

# Нажмем на чекбокс, если он уже не активирован
if not checkbox.is_selected():
    checkbox.click()

time.sleep(1)

search_input = driver.find_element(By.ID, "funds-search")
search_input.clear()
search_input.send_keys("теплый дом")
search_input.send_keys(Keys.RETURN)

wait = WebDriverWait(driver, 10)
submit_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.np-button.np-button_block.np-button_big.b-funds-filter__btn-submit.js-funds-filter-submit')))

submit_button.click()

time.sleep(5)

while True:
    try:
        load_more_button = driver.find_element(By.CSS_SELECTOR, 'button.b-funds__more-button.np-button.np-button_more.np-button_big.np-button_block.np-button_border-black.js-load-more')
        if load_more_button.is_displayed():
            load_more_button.click()
            time.sleep(10)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            print("*")
            
            links = driver.find_elements(By.CSS_SELECTOR, 'a.np-card__link')
            for link in links:
                href = link.get_attribute('href')
                print("https://nuzhnapomosh.ru" + href)
        else:
            break
    except NoSuchElementException:
        break

driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=C:/Users/cherp/AppData/Local/Google/Chrome/User Data/Default')

driver = webdriver.Chrome(options=options)

driver.get("https://nuzhnapomosh.ru/funds/teplyi_dom_1067847576037")

fund_info = driver.find_element(By.CLASS_NAME, 'b-fund-single-info')

info_items = fund_info.find_elements(By.CLASS_NAME, 'b-fund-single-info__item')

info_dict = {}

for item in info_items:
    title = item.find_element(By.CLASS_NAME, 'b-fund-single-info__title').text
    info = item.find_element(By.CLASS_NAME, 'b-fund-single-info__info').text
    info_dict[title] = info

for key, value in info_dict.items():
    print(f"{key}: {value}")

driver.quit()


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm

options = webdriver.ChromeOptions()
options.add_argument('--user-data-dir=C:/Users/cherp/AppData/Local/Google/Chrome/User Data/Default')

driver = webdriver.Chrome(options=options)

with open("Ссылки_фондыНужна помощь.txt", "r") as file:
    links = file.read().splitlines()

data = []

for link in tqdm(links):
    driver.get(link)
    fund_info = driver.find_element(By.CLASS_NAME, 'b-fund-single-info')
    info_items = fund_info.find_elements(By.CLASS_NAME, 'b-fund-single-info__item')

    info_dict = {}

    for item in info_items:
        title = item.find_element(By.CLASS_NAME, 'b-fund-single-info__title').text
        info = item.find_element(By.CLASS_NAME, 'b-fund-single-info__info').text
        info_dict[title] = info

    data.append(info_dict)

driver.quit()

df = pd.DataFrame(data)

df['Ссылка'] = links
cols = df.columns.tolist()
cols = ['Ссылка'] + [col for col in cols if col != 'Ссылка']
df = df[cols]

df.to_excel("ФондыНужнапомощь_реквизиты.xlsx", index=False)


In [ ]:
df

In [ ]:
import pandas as pd

df1 = pd.read_excel("ФондыНужнапомощь_реквизиты.xlsx")
df2 = pd.read_excel("Информация_фондыСберВместе.xlsx")

merged_df = df2.merge(df1, on='СсылкаНужнапомощь', how='left')

desired_columns_order = [
    'Ссылка на страницу', 'Название фонда', 'Собрано фондом', 'Человек помогло фонду',
    'Завершённых сборов', 'СсылкаНужнапомощь', 'Название организации', 'Статус',
    'Руководитель', 'ИНН', 'КПП', 'ОГРН'
]

merged_df = merged_df[desired_columns_order]

merged_df.to_excel("Объединенные_фонды.xlsx", index=False)
